In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This is a notebook that illustrates how to run the end-to-end (e2e) workflow. Determining suitable parameters for the workflow are for operational considerations are explained in the ops library [`dist-s1-enumerator`](https://github.com/opera-adt/dist-s1-enumerator) - specifically, see this [notebook](https://github.com/opera-adt/dist-s1-enumerator/blob/dev/notebooks/A__Staging_Inputs_for_One_MGRS_Tile.ipynb).

In [13]:
from pathlib import Path

from dist_s1.workflows import (
    run_burst_disturbance_workflow,
    run_dist_s1_localization_workflow,
    run_dist_s1_workflow,
    run_disturbance_merge_workflow,
    run_dist_s1_packaging_workflow,
    run_despeckle_workflow
)

In [14]:
## Example 0

# mgrs_tile_id = '10SGD'
# post_date = '2025-01-02'
# track_number = 137
# dst_dir = Path('out')
# memory_strategy = 'high'


## Example 1 - Los Angeles Wildfire
mgrs_tile_id = '11SLT'
post_date = '2025-01-21'
track_number = 71
dst_dir = Path('los-angeles')
memory_strategy = 'high'
confirmation = False
interpolation_method='bilinear'

This takes some time as it not only generates the runconfig but also localizes the necessary data. It will not overwrite files that have been previously downloaded.

Few additional notes:

1. The runconfig can be serialized to yml and then read from that file.
2. The runconfig manages all the paths for the workflows from the initial set of inputs. It's data model is messy and can be confusing. But that's the workhorse of this library.

In [15]:
run_config = run_dist_s1_localization_workflow(
    mgrs_tile_id,
    post_date,
    track_number,
    post_date_buffer_days=1,
    dst_dir=dst_dir,
    input_data_dir=dst_dir,
    confirmation=confirmation,
    interpolation_method='bilinear',
)


Using multi-window lookback strategy


/Users/cmarshak/bekaert-team/dist-s1/src/dist_s1/data_models/runconfig_model.py:650: UserWarning: CUDA and mps do not support multiprocessing; setting n_workers_for_norm_param_estimation to 1


In [20]:
run_config.confirmation = True

ValidationError: 1 validation error for RunConfigData
  Value error, prior_dist_s1_product must be provided when confirmation is True [type=value_error, input_value=RunConfigData(device='mps..., algo_config_path=None), input_type=RunConfigData]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [17]:
run_config.prior_dist_s1_product = 'foo'

ValidationError: 1 validation error for RunConfigData
prior_dist_s1_product
  Input should be a valid dictionary or instance of ProductDirectoryData [type=model_type, input_value='foo', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [6]:
if run_config.apply_despeckling:
    run_despeckle_workflow(run_config)

Despeckling and serializing RTC S1 files: 100%|█| 352/352 [0


In [7]:
# Compute disturbance per burst and all possible lookbacks
run_burst_disturbance_workflow(run_config)

Chips Traversed: 100%|███| 656/656 [00:01<00:00, 535.24it/s]
/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/torch/nn/functional.py:5561: UserWarning: The operator 'aten::col2im' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1746257056484/work/aten/src/ATen/mps/MPSFallback.mm:14.)
  return torch._C._nn.col2im(
/Users/cmarshak/bekaert-team/dist-s1/src/dist_s1/dist_processing.py:132: RuntimeWarning: All-NaN slice encountered
  metric = np.nanmax(z_score_per_channel, axis=0)
Burst disturbance: 100%|████| 16/16 [00:46<00:00,  2.88s/it]


In [8]:
# Merge the burst-wise products
run_disturbance_merge_workflow(run_config)

In [9]:
run_dist_s1_packaging_workflow(run_config)

No confirmation requested, skipping confirmation step


/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/numpy/_core/_asarray.py:127: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=None, subok=subok)
